In [1]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#wrangling
import pandas as pd
import numpy as np

#explore
import scipy.stats as stats

#visuals
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


#default pandas decimal display formatting
pd.options.display.float_format='{:20,.2f}'.format

import env
import acquire
# import prepare
# import summarize

### Acquire & Summarize

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

    - Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
    - Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
    - Only include properties that include a latitude and longitude value.

Acquired zillow data using acquire.py

In [2]:
df = acquire.get_zillow_data()

In [3]:
# df.transactiondate = pd.to_datetime(df.transactiondate, format='%Y-%m-%d')
# df = df.sort_values("transactiondate").drop_duplicates('parcelid',keep='last') 

In [4]:
df.head()

,county,tax_rate,id,parcelid,airconditioningtypeid,airconditioningdesc,architecturalstyletypeid,architecturalstyledesc,basementsqft,bathroomcnt,...,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,typeconstructiontypeid,typeconstructiondesc,censustractandblock,transactiondate,logerror,transactions
0,Ventura,0.01,1387261,17052889,nan,None,nan,None,nan,1.00,...,"376,000.00","5,672.48",None,nan,nan,None,"61,110,010,023,006.00",2017-01-01,0.06,1
1,Ventura,0.01,1447245,17143294,nan,None,nan,None,nan,2.00,...,"132,424.00","3,508.10",None,nan,nan,None,"61,110,053,062,009.00",2017-01-01,-0.02,1
2,Ventura,0.01,43675,17110996,nan,None,nan,None,nan,2.50,...,"99,028.00","2,204.84",None,nan,nan,None,"61,110,050,041,163.00",2017-01-02,0.01,1
3,Ventura,0.01,1327940,17153340,nan,None,nan,None,nan,3.50,...,"522,030.00","12,738.54",None,nan,nan,None,"61,110,036,093,023.00",2017-01-02,-0.10,1
4,Ventura,0.01,1431120,17153706,nan,None,nan,None,nan,2.50,...,"239,000.00","8,442.56",None,nan,nan,None,"61,110,036,093,015.00",2017-01-02,0.03,1


2. Summary of zillow data (summary stats, info, dtypes, shape, distributions)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 72 columns):
county                          77381 non-null object
tax_rate                        77375 non-null float64
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
airconditioningdesc             24953 non-null object
architecturalstyletypeid        206 non-null float64
architecturalstyledesc          206 non-null object
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingclassdesc               15 non-null object
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
decktypeid                      

In [10]:
df.describe()

,tax_rate,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,fireplaceflag,structuretaxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,typeconstructiontypeid,censustractandblock,logerror,transactions
count,"77,375.00","77,381.00","77,381.00","24,953.00",206.00,50.00,"77,381.00","77,381.00",15.00,"49,672.00",...,172.00,"77,269.00","77,381.00","77,379.00","77,376.00","2,886.00",222.00,"77,137.00","77,381.00","77,381.00"
mean,0.01,"1,495,138.90","13,007,150.51",1.81,7.39,679.72,2.30,3.05,3.93,6.53,...,1.00,"189,314.53","2,016.00","301,095.41","5,995.53",14.09,6.04,"60,496,733,646,264.24",0.02,1.00
std,0.01,"860,907.05","3,481,345.65",2.97,2.73,689.70,1.00,1.14,0.26,1.72,...,0.00,"230,087.42",0.00,"492,596.03","7,622.84",2.19,0.56,"1,535,241,981,138.57",0.17,0.05
min,0.00,349.00,"10,711,855.00",1.00,2.00,38.00,0.00,0.00,3.00,1.00,...,1.00,44.00,"2,016.00",161.00,19.92,3.00,4.00,"60,371,011,101,000.00",-4.66,1.00
25%,0.01,"752,070.00","11,538,305.00",1.00,7.00,273.00,2.00,2.00,4.00,6.00,...,1.00,"84,265.00","2,016.00","85,504.00","2,715.63",14.00,6.00,"60,373,109,005,002.00",-0.02,1.00
50%,0.01,"1,497,932.00","12,531,568.00",1.00,7.00,515.00,2.00,3.00,4.00,6.00,...,1.00,"136,499.00","2,016.00","203,372.00","4,450.69",15.00,6.00,"60,376,032,003,008.00",0.01,1.00
75%,0.01,"2,240,535.00","14,211,831.00",1.00,7.00,796.50,3.00,4.00,4.00,8.00,...,1.00,"218,787.00","2,016.00","366,796.50","6,927.79",15.00,6.00,"60,590,423,251,008.00",0.04,1.00
max,0.82,"2,982,274.00","167,689,317.00",13.00,21.00,"3,560.00",18.00,16.00,4.00,12.00,...,1.00,"11,421,790.00","2,016.00","48,952,198.00","586,639.30",99.00,13.00,"483,030,105,084,015.00",5.26,3.00


In [11]:
df.dtypes

county                           object
tax_rate                        float64
id                                int64
parcelid                          int64
airconditioningtypeid           float64
airconditioningdesc              object
architecturalstyletypeid        float64
architecturalstyledesc           object
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingclassdesc                object
buildingqualitytypeid           float64
calculatedbathnbr               float64
calculatedfinishedsquarefeet    float64
decktypeid                      float64
finishedfloor1squarefeet        float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
state                            object


In [19]:
pd.DataFrame(df.columns)

,0
0,county
1,tax_rate
2,id
3,parcelid
4,airconditioningtypeid
5,airconditioningdesc
6,architecturalstyletypeid
7,architecturalstyledesc
8,basementsqft
9,bathroomcnt


In [20]:
df.shape

(77381, 72)

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

- Number of rows with missing values

In [20]:
number_missing = df.isnull().sum()

- percent of total rows that have missing values

In [21]:
pct_missing = (df.isnull().sum())/df.shape[0]

In [25]:
rows_missing_df = pd.DataFrame({'num_rows_missing': number_missing, 'pct_rows_missing': pct_missing})

In [26]:
def nulls_by_col(df):
    number_missing = df.isnull().sum()
    pct_missing = (df.isnull().sum())/df.shape[0]
    rows_missing_df = pd.DataFrame({'num_rows_missing': number_missing, 'pct_rows_missing': pct_missing})
    return rows_missing_df

In [28]:
rows_missing_df

,num_rows_missing,pct_rows_missing
county,0,0.00
tax_rate,6,0.00
id,0,0.00
parcelid,0,0.00
airconditioningtypeid,52428,0.68
airconditioningdesc,52428,0.68
architecturalstyletypeid,77175,1.00
architecturalstyledesc,77175,1.00
basementsqft,77331,1.00
bathroomcnt,0,0.00


4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [29]:
df.isnull().sum(axis=1)

0        33
1        33
2        33
3        32
4        32
5        33
6        33
7        34
8        33
9        33
10       32
11       33
12       33
13       34
14       31
15       32
16       33
17       33
18       32
19       29
20       32
21       30
22       29
23       34
24       34
25       31
26       32
27       33
28       33
29       33
         ..
77351    31
77352    34
77353    29
77354    33
77355    32
77356    34
77357    34
77358    35
77359    29
77360    35
77361    35
77362    34
77363    35
77364    34
77365    37
77366    35
77367    34
77368    29
77369    37
77370    33
77371    38
77372    35
77373    37
77374    37
77375    36
77376    38
77377    32
77378    36
77379    34
77380    37
Length: 77381, dtype: int64

In [32]:
df.shape

(77381, 72)

In [ ]:
def nulls_by_row(df):
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100
    rows_missing = pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing}).reset_index().groupby(['num_cols_missing','pct_cols_missing']).count().rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing